In [1]:
print("transformer")

transformer


词嵌入，embeddings，让文本变为数字，embeddings越好模型就会越好。
相似的单词会赋予相似的数据，这样距离比较近。

attention计算出一个句子中每个单词之间的相关性。
三种表示相关性的方法
1. 点积（向量之间，相同坐标轴的值相乘然后求和）
2. 余弦相似度（计算两个向量之间的夹角，可以用点积/模长）
3. 比例点积（点积/sqrt(维度)，利用维度大小对点积进行缩放）

In [2]:
import numpy as np

text1 = '我喜欢的水果是橙子和苹果'
text2 = '相比苹果我更加喜欢国产的华为'

In [5]:
import spacy

nlp = spacy.load('zh_core_web_sm')

In [6]:
doc = nlp(text1)
doc

我喜欢的水果是橙子和苹果

In [7]:
emd_dim = 10

In [13]:
dics = {}
for token in doc:
    dics[token.text] = token.vector[:emd_dim]
dics

{'我': array([ 1.4686487 , -0.72315717, -1.2931682 ,  0.24932906,  2.3291414 ,
         1.7527952 , -0.25596786,  0.8271904 , -0.51447475, -1.0562801 ],
       dtype=float32),
 '喜欢': array([ 0.51233214,  0.6927922 ,  0.00741255,  1.460608  , -0.0323258 ,
        -0.02877208, -0.2949775 ,  0.31405067, -0.70030355,  0.7001613 ],
       dtype=float32),
 '的': array([ 3.3848634 ,  2.5763922 , -1.3831755 ,  1.7510111 , -2.0771494 ,
         2.153421  , -0.79614574, -0.2623603 , -0.19018924,  0.55982596],
       dtype=float32),
 '水果': array([ 0.93435514, -2.026781  , -0.7905892 , -1.1106956 ,  1.0638433 ,
        -0.62800986,  3.8172576 , -1.0496758 , -0.55736905, -0.9285867 ],
       dtype=float32),
 '是': array([ 0.66781914,  0.3840046 ,  0.05266917, -0.38295984, -1.6400721 ,
        -1.5146024 ,  1.938287  , -1.456138  ,  0.11141145, -2.9090867 ],
       dtype=float32),
 '橙子': array([ 1.0052307 , -1.7132874 , -2.3313982 , -0.06937416,  1.9805148 ,
        -2.4350555 ,  0.4545729 , -2.1072867

In [14]:
import pandas as pd
X = pd.DataFrame(dics)
X.T

,0,1,2,3,4,5,6,7,8,9
我,1.468649,-0.723157,-1.293168,0.249329,2.329141,1.752795,-0.255968,0.827190,-0.514475,-1.056280
喜欢,0.512332,0.692792,0.007413,1.460608,-0.032326,-0.028772,-0.294977,0.314051,-0.700304,0.700161
的,3.384863,2.576392,-1.383175,1.751011,-2.077149,2.153421,-0.796146,-0.262360,-0.190189,0.559826
水果,0.934355,-2.026781,-0.790589,-1.110696,1.063843,-0.628010,3.817258,-1.049676,-0.557369,-0.928587
是,0.667819,0.384005,0.052669,-0.382960,-1.640072,-1.514602,1.938287,-1.456138,0.111411,-2.909087
橙子,1.005231,-1.713287,-2.331398,-0.069374,1.980515,-2.435055,0.454573,-2.107287,-0.501081,-1.240167
和,0.915805,-1.751494,1.843134,-0.005472,-0.945925,-0.718926,1.925642,3.552628,2.092016,-0.643384
苹果,-1.193919,-0.804299,0.572513,-1.500060,0.772428,-1.325299,3.144667,-0.644317,-0.995969,0.586208


### 初始化q,k,v三个矩阵

In [15]:
d_k = 6
Wq = np.random.randn(emd_dim, d_k)
Wq

array([[-0.52372773,  1.42515819,  0.58178993, -0.20832455, -1.32264271,
         0.81341617],
       [ 0.14896661,  0.79445674, -1.36983142,  1.68399155, -0.32608763,
         0.69076801],
       [ 1.72562425,  0.71077768, -0.17516934,  1.80495265,  0.51597991,
         1.39529745],
       [-0.7176602 ,  0.7503454 , -0.55476664, -1.10961229, -0.3122623 ,
        -1.0889995 ],
       [-0.6368331 , -1.43705958, -1.16865934, -0.09098364, -0.14238896,
        -0.15765252],
       [-0.04866623,  1.59708744, -0.0901558 , -0.06501652, -0.47633378,
        -0.13415822],
       [-1.29611754,  1.53209047,  0.09147977,  0.46758101, -0.4769985 ,
         0.19116311],
       [ 0.72542873, -0.28826896, -0.27210867, -1.29977802, -1.2991644 ,
        -0.32932466],
       [ 0.11993163,  0.43982556, -1.56257386, -0.02001143,  1.16770436,
         0.34689051],
       [-0.32928176, -1.36635593,  1.73988999,  0.35845052,  0.35405941,
         0.02494866]])

In [22]:
Wk = np.random.randn(emd_dim, d_k)
Wv = np.random.randn(emd_dim, d_k)

### 计算得到q,k,v矩阵

In [16]:
X.T.shape

(8, 10)

In [17]:
Wq

array([[-0.52372773,  1.42515819,  0.58178993, -0.20832455, -1.32264271,
         0.81341617],
       [ 0.14896661,  0.79445674, -1.36983142,  1.68399155, -0.32608763,
         0.69076801],
       [ 1.72562425,  0.71077768, -0.17516934,  1.80495265,  0.51597991,
         1.39529745],
       [-0.7176602 ,  0.7503454 , -0.55476664, -1.10961229, -0.3122623 ,
        -1.0889995 ],
       [-0.6368331 , -1.43705958, -1.16865934, -0.09098364, -0.14238896,
        -0.15765252],
       [-0.04866623,  1.59708744, -0.0901558 , -0.06501652, -0.47633378,
        -0.13415822],
       [-1.29611754,  1.53209047,  0.09147977,  0.46758101, -0.4769985 ,
         0.19116311],
       [ 0.72542873, -0.28826896, -0.27210867, -1.29977802, -1.2991644 ,
        -0.32932466],
       [ 0.11993163,  0.43982556, -1.56257386, -0.02001143,  1.16770436,
         0.34689051],
       [-0.32928176, -1.36635593,  1.73988999,  0.35845052,  0.35405941,
         0.02494866]])

In [ ]:
np.dot(X.T, Wq)     # 使用的数值

array([[-3.63798639,  0.82507597, -2.22915309, -6.02356487, -5.54564968,
        -2.50929662],
       [-0.88295423,  0.57513534,  0.7778772 , -0.82372748, -2.17465194,
        -1.06127851],
       [-3.17999937, 11.63301116,  1.2140601 , -0.58383517, -6.61080771,
         0.61696922],
       [-7.47551512,  2.96889052,  2.7780784 , -1.03070314, -1.92508862,
         0.24184897],
       [-1.40596832,  8.35807565, -2.5430656 ,  3.0292028 ,  0.16054815,
         2.57690398],
       [-7.66736974, -5.59476118,  0.52382864, -4.73049293,  0.42239839,
        -2.95269107],
       [ 3.62539435,  5.15615582, -1.3963773 , -3.66435849, -2.52919198,
         2.2661036 ],
       [-2.71320003, -2.52115223,  3.39502903,  4.14593567,  1.50814471,
         1.44414552]])

In [20]:
Q = X.T @ Wq    # 原有的索引不变
Q

,0,1,2,3,4,5
我,-3.637986,0.825076,-2.229153,-6.023565,-5.545650,-2.509297
喜欢,-0.882954,0.575135,0.777877,-0.823727,-2.174652,-1.061279
的,-3.179999,11.633011,1.214060,-0.583835,-6.610808,0.616969
水果,-7.475515,2.968891,2.778078,-1.030703,-1.925089,0.241849
是,-1.405968,8.358076,-2.543066,3.029203,0.160548,2.576904
橙子,-7.667370,-5.594761,0.523829,-4.730493,0.422398,-2.952691
和,3.625394,5.156156,-1.396377,-3.664358,-2.529192,2.266104
苹果,-2.713200,-2.521152,3.395029,4.145936,1.508145,1.444146


In [23]:
K = X.T @ Wk
V = X.T @ Wv

### 相似矩阵

In [24]:
Q @ K.T

,我,喜欢,的,水果,是,橙子,和,苹果
我,51.829180,-1.581374,43.564214,95.338670,57.890565,103.045250,-68.826156,12.547107
喜欢,14.987918,2.479846,16.025146,21.795331,19.358592,24.436332,-16.407843,-0.758046
的,48.811483,24.103247,147.995686,7.192115,27.766749,49.382609,-105.913819,-76.958966
水果,13.227532,10.595085,54.129361,-2.681422,1.660940,4.533103,-60.768168,-21.291000
是,14.570448,16.633769,84.559286,-50.031597,-38.304909,-6.350580,-45.187006,-67.794450
橙子,13.398275,-5.310128,-37.092884,55.470576,9.915891,29.256097,-12.080373,49.352851
和,4.918160,-1.641768,57.874638,17.755230,32.593088,19.589774,-46.646841,-22.412547
苹果,-29.394428,-1.484698,-27.573754,-50.445874,-23.362221,-49.303022,16.437893,-0.024499


### 点积缩放

In [26]:
df_QK = Q @ K.T / np.sqrt(d_k)

### SoftMax

In [29]:
df_QK.iloc[0]

我     21.159174
喜欢    -0.645593
的     17.785016
水果    38.921849
是     23.633724
橙子    42.068047
和    -28.098161
苹果     5.122335
Name: 我, dtype: float64

In [30]:
for i in range(len(df_QK)):
    exp_v = np.exp(df_QK.iloc[i])
    softmax = exp_v / np.sum(exp_v)
    df_QK.iloc[i] = softmax

df_QK

,我,喜欢,的,水果,是,橙子,和,苹果
我,7.963446e-10,2.700293e-19,2.727237e-11,4.124135e-02,9.457679e-09,9.587586e-01,3.227843e-31,8.637547e-17
喜欢,1.390192e-02,8.421589e-05,2.123124e-02,2.238871e-01,8.279334e-02,6.580796e-01,3.772308e-08,2.245514e-05
的,2.598023e-18,1.081118e-22,1.000000e+00,1.085250e-25,4.824051e-22,3.280223e-18,9.590693e-46,1.304802e-40
水果,5.598949e-08,1.911497e-08,9.999999e-01,8.461536e-11,4.981383e-10,1.609105e-09,4.252756e-21,4.245917e-14
是,3.899245e-13,9.053306e-13,1.000000e+00,1.371006e-24,1.645056e-22,7.615225e-17,9.907980e-24,9.720437e-28
橙子,3.207973e-08,1.546077e-11,3.582195e-17,9.239507e-01,7.741127e-09,2.078846e-05,9.746909e-13,7.602847e-02
和,4.081342e-10,2.803677e-11,9.999668e-01,7.705758e-08,3.292814e-05,1.629596e-07,2.939878e-19,5.821959e-15
苹果,7.466533e-09,6.630003e-04,1.570090e-08,1.382605e-12,8.762523e-08,2.204579e-12,9.981335e-01,1.203392e-03


### V矩阵

In [31]:
V

,0,1,2,3,4,5
我,7.234629,9.622294,2.218120,-0.703184,3.909720,4.335888
喜欢,-1.223946,0.246356,-1.107621,4.182677,1.664727,-3.450533
的,-8.064973,8.461239,-2.503840,14.981975,-4.553541,-10.127521
水果,-1.905798,10.619286,12.579398,-9.017163,1.721978,5.323443
是,0.914865,2.030718,8.664424,-3.503416,-4.630994,3.771061
橙子,1.238529,4.092301,5.810728,-2.737049,5.542250,1.094313
和,8.785840,10.118650,1.128463,-7.898524,-12.680482,6.012545
苹果,-5.010697,-2.421290,8.995538,-10.536218,2.217835,3.360905


In [33]:
attention = df_QK @ V
attention

,0,1,2,3,4,5
我,1.108852,4.361483,6.089877,-2.996049,5.384697,1.268728
喜欢,0.393242,5.552089,7.335429,-3.801657,3.607220,2.069257
的,-8.064973,8.461239,-2.503840,14.981975,-4.553541,-10.127521
水果,-8.064972,8.461239,-2.503840,14.981974,-4.553541,-10.127520
是,-8.064973,8.461239,-2.503840,14.981975,-4.553541,-10.127521
橙子,-2.141793,9.627695,12.306782,-9.132524,1.759757,5.174146
和,-8.064675,8.461027,-2.503470,14.981362,-4.553542,-10.127060
苹果,8.762600,10.097013,1.136448,-7.893687,-12.653042,6.003079
